In [1]:
import sqlite3
import pandas as pd
import yfinance as yf
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV, train_test_split

# Data Model Implementation (25 points)

In [24]:
# The data is cleaned, normalized, and standardized prior to modeling (5 points)
tesla = yf.Ticker('TSLA')
hist = tesla.history('1y')
hist[:5]

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2023-09-14 00:00:00-04:00,271.320007,276.709991,270.420013,276.040009,107709800,0.0,0.0
2023-09-15 00:00:00-04:00,277.549988,278.980011,271.000000,274.390015,133422800,0.0,0.0
2023-09-18 00:00:00-04:00,271.160004,271.440002,263.760010,265.279999,101543300,0.0,0.0
2023-09-19 00:00:00-04:00,264.350006,267.850006,261.200012,266.500000,103704000,0.0,0.0
2023-09-20 00:00:00-04:00,267.040009,273.929993,262.459991,262.589996,122514600,0.0,0.0


In [42]:
# dropped Volume, Dividends, and Stock Splits
df = hist.drop(['Volume','Dividends','Stock Splits'],axis=1)

# changing date format
df.index = df.index.strftime('%Y-%m-%d %a')

# Dropping the first two rows to start on a Monday
df = df[2:]

In [43]:
# The model utilizes data retrieved from SQL or Spark (5 points)
db = sqlite3.connect('stock_db.sqlite')
df.to_sql('tesla',db, if_exists='replace')

df = pd.read_sql('SELECT * FROM tesla', db, index_col='Date')
df[:5]

,Open,High,Low,Close
Date,,,,
2023-09-18 Mon,271.160004,271.440002,263.760010,265.279999
2023-09-19 Tue,264.350006,267.850006,261.200012,266.500000
2023-09-20 Wed,267.040009,273.929993,262.459991,262.589996
2023-09-21 Thu,257.850006,260.859985,254.210007,255.699997
2023-09-22 Fri,257.399994,257.790009,244.479996,244.880005


In [ ]:
# A Python script initializes, trains, and evaluates a model (10 points)
